In [52]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from category_encoders import OrdinalEncoder, TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,classification_report, roc_curve, confusion_matrix, ConfusionMatrixDisplay
import pickle
import sqlite3

DATA_PATH = os.path.join(os.getcwd(), 'data', '_github-AAC_accidents_tagged_data.xlsx') 

df = pd.read_excel(DATA_PATH, sheet_name='data')


In [53]:
def process(df):
    #Publication Year float to int dtype.
    df.dropna(subset=['Publication Year'], inplace=True)
    df['Publication Year'] = df['Publication Year'].astype(int)

    #Injury level conversion.
    # 1: Minor, 2: Serious, 3: Deadly.
    df['injury_level'] = [0]*df.shape[0]
    injury_level_idx = df.columns.get_loc('injury_level')
    df.iloc[np.array(df.loc[:,'Minor'] == 1), injury_level_idx] = 1
    df.iloc[np.array(df.loc[:,'Serious'] == 1), injury_level_idx] = 2
    df.iloc[np.array(df.loc[:,'Deadly'] == 1), injury_level_idx] = 3
    
    #Columns to Drop
    drop_idx = ['ID', 'Accident Title', 'Text', 'Tags Applied', 'Search Column\n\n', 
        'COUNT OF TAGS', '<15', '15-20', '21-25', '26-30','31-35', '36-50', '51-75', '>75', 'ID',
        'Head / Brain Injury', 'Deadly', 'Serious','Minor']
    df.drop(drop_idx, axis=1, inplace=True)

    #Dropping rows with 'Y' value in month columns.
    #Month Columns, Jan-Dec
    df.replace('Y', 1, inplace=True)

    #Drop dubplicated data rows.
    df.drop_duplicates(inplace=True)


    month_cols = ['January','February', 'March', 'April', 'May', 'June', 'July', 'August','September', 'October', 'November', 'December']
    exp_cols = ['No/Little', 'Moderate','Experienced', 'Unknown']
    climbing_type_cols = ['Descent', 'Roped', 'Trad Climbing', 'Sport',
       'Top-Rope', 'Aid & Big Wall Climbing', 'Pendulum', 'Unroped ', 'Solo',
       'Climbing Alone', 'Bouldering', 'Non-climbing','Alpine/Mountaineering']
    
    #Aggregated column for easy sql visualization
    modify_dict = {'Month':month_cols, 'Experience':exp_cols, 'Type of Climbing':climbing_type_cols}
    for agg in modify_dict.keys():
        df[agg] = [0]*df.shape[0]
        idx = df.columns.get_loc(agg)
        for col_name in modify_dict[agg]:
            df.iloc[np.array(df.loc[:,col_name] == 1), idx] = col_name

    #Dropping rows with missing values in experience or climbing type columns.
    df['Experience'].replace(0, 'Unknown', inplace=True)
    df['Type of Climbing'].replace(0, 'Unknown', inplace=True)

    #Replacing all NaN with int 0.
    df.fillna(0,inplace=True)

    df.iloc[:, :-3] = df.iloc[:,:-3].astype(int)

    return df

df_clean = process(df.copy())

In [54]:
month_cols = ['January',
       'February', 'March', 'April', 'May', 'June', 'July', 'August',
       'September', 'October', 'November', 'December']
exp_cols = ['No/Little', 'Moderate','Experienced', 'Unknown']
climbing_type_cols = ['Descent', 'Roped', 'Trad Climbing', 'Sport',
       'Top-Rope', 'Aid & Big Wall Climbing', 'Pendulum', 'Unroped ', 'Solo',
       'Climbing Alone', 'Bouldering', 'Non-climbing','Alpine/Mountaineering']
alpine_ice_factors_cols = ['Piton/Ice Screw', 'Ascent Illness', 'Crampon Issues', 'Ice Climbing',
       'Glissading', 'Ski-related ', 'Poor Position']
natural_factors_cols =['Poor Cond/Seasonal Risk', 'Avalanche',
       'Cornice / Snow Bridge Collapse', 'Bergschrund',
       'Crevasse / Moat / Berschrund', 'Icefall / Serac / Ice Avalanche',
       'Exposure', 'Non-Ascent Illness', 'Visibility', 'Severe Weather',
       'Wildlife', 'Natural Rockfall']

In [55]:
cnx = sqlite3.connect('data/AAC_climbing_accidents.db')
cursor = cnx.cursor()
cursor.execute('DROP TABLE IF EXISTS accident_table')
cursor.execute("CREATE TABLE accident_table(id INTEGER PRIMARY KEY AUTOINCREMENT)")

cnx.commit() 

df_clean.to_sql('accident_table', cnx, if_exists='replace', index=False)

df_cw = pd.read_sql_query("SELECT * FROM accident_table", cnx)

In [56]:
target = ['injury_level']
drop = [target[0], 'Month', 'Experience', 'Type of Climbing']

train, test = train_test_split(df_cw, train_size=0.9, test_size=0.1, random_state=3)
train, val = train_test_split(df_cw, train_size=0.80, test_size=0.2, random_state=3)

X_train = train.drop(columns=drop)
y_train = train[target]
X_val = val.drop(columns=drop)
y_val = val[target]
X_test = test.drop(columns=drop)
y_test = test[target]

In [57]:
#Mode class : Serious
print("Injury Outcome Counts: ")
print(df_clean['injury_level'].value_counts())
counter = df_clean['injury_level'].value_counts()

#Baseline Model Accuracy
baseline_accuracy = counter.iloc[0]/counter.sum()
print ('\nBaseline Model Accuracy:' , baseline_accuracy)

Injury Outcome Counts: 
2    1771
3     561
1     219
0     190
Name: injury_level, dtype: int64

Baseline Model Accuracy: 0.6461145567311201


In [58]:
pipe = make_pipeline(
    RandomForestClassifier(random_state=42,
                  n_jobs=-1,
                  n_estimators=1000,
                  max_depth=10,
                 )
)
pipe.fit(X_train, np.ravel(y_train));

with open('model.pkl','wb') as f:
    pickle.dump(pipe, f)

In [65]:
X_test.iloc[0]

Publication Year    2013
No/Little              1
Moderate               0
Experienced            0
Unknown                0
                    ... 
August                 0
September              0
October                0
November               0
December               0
Name: 1910, Length: 78, dtype: int64

In [73]:
pipe.predict(X_test.iloc[0].values.reshape(1,-1))[0]

/Users/Chanhyung/opt/anaconda3/envs/section3_prj/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


2